## EBAC - Profissão Cientista de Dados<br><br> Módulo 23: Combinação de Modelos I 


## Exercício 2


**1) Monte um passo a passo para o algoritmo Random Forest**

**Passo 1 - Amostragem com reposição (Bootstrap)**

- Criamos um número x de subconjuntos de dados a partir do conjunto de dados original, através de uma amostragem com reposição, ou seja, cada elemento de cada subconjunto de dados será escolhido aleatoriamente na base de dados original, tendo a possibilidade de ser selecionado novamente.

**Passo 2 - Feature Selection**

 - Em cada subconjunto de dados, devemos fazer uma seleção aleatória de variáveis (features), ou seja, cada subconjunto de dados deverá ter apenas algumas das colunas do conjunto de dados original, escolhidas aleatoriamente. 
 
 - O número de de colunas que os subconjuntos deverão ter será igual a:<br><br> <center>$\sqrt{p}$  - para árvores de classificação;e</center></br><br><center>$\frac{p}{3}$  - para árvores de regressão.</center></br> <br>Onde p é o número de colunas do conjunto de dados original</br>
  
**Passo 3 - Modelagem das Árvores de Decisão**

- Treinamos uma árvore separada para cada subconjunto de treinamento gerado na etapa anterior.

**Passo 3 - Agregação (Agreggating)**

- Combinamos as previsões de cada uma das árvores para obter uma previsão final, selecionando o valor predito com maior frequência das árvores de decisões ou a média dos valores preditos das árvores de classificação.




**2) Explique com suas palavras o Random Forest**

O Random Forest é um método de aprendizado de máquina que combina várias árvores de decisão para realizar tarefas de classificação ou regressão. Ele utiliza a técnica de amostragem com reposição (bootstrap) para criar subconjuntos de dados a partir do conjunto de treinamento original onde cada subconjunto criado também possui um subconjunto aleatório das variáveis do conjunto de dados original.Uma árvore é treinada com cada um dos subconjuntos e o resultado agregado atráves da escola do resultado de maior frequência ou da média dos valores preditos. 

**3) Qual a diferença entre Bagging e Random Forest?**

O Random Forest é uma extensão o do Bagging, agregando um elemento adicional de aleatoriedade. A diferença é que, além de fazer uma amostragem com reposição dos dados, criando subconjuntos de dados, também são amostradas aleatoriamente um subconjunto de variáveis de forma que cada subconjunto de dados tenha apenas algumas das variáveis do conjunto de dados de origem, escolhidas aleatoriamente.  

**4) (Opcional) Implementar em python o código do Random Forest**

In [1]:
#importando as bibliotecas

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier


In [2]:
#Criando Dataframe Treino
# gerando 6 variáveis explicativa x1, cada uma com 5 números aleatórios entre 1 e 10
x = [np.random.randint(1, 11, size=5) for i in range(1,7)]

# y -  variável resposta
# Gerar 5 números aleatórios entre 0 e 1
y = np.random.randint(0, 2, size=5)

# Criar DataFrame
df_treino = pd.merge(pd.DataFrame(x).T,pd.DataFrame(y),left_index = True, right_index=True)
df_treino.columns = [ 'x1','x2','x3','x4','x5','x6','y']

#exibir Dataframe
df_treino


,x1,x2,x3,x4,x5,x6,y
0,3,8,7,3,6,10,1
1,4,6,5,4,5,5,1
2,9,3,3,6,10,4,0
3,9,7,3,4,1,1,0
4,3,4,1,3,6,5,0


In [3]:
#Criando Dataframe Teste

# gerando 6 variáveis explicativa x1, cada uma com 3 números aleatórios entre 1 e 10
x = [np.random.randint(1, 11, size=3) for i in range(1,7)]

# y -  variável resposta
# Gerar 3 números aleatórios entre 0 e 1
y = np.random.randint(0, 2, size=3)

# Criar DataFrame
df_teste = pd.merge(pd.DataFrame(x).T,pd.DataFrame(y),left_index = True, right_index=True)
df_teste.columns = [ 'x1','x2','x3','x4','x5','x6','y']

#exibir Dataframe
df_teste


,x1,x2,x3,x4,x5,x6,y
0,4,4,5,3,5,1,1
1,1,6,1,6,4,5,0
2,9,8,6,4,8,3,0


In [4]:
# Criando funções para cada passo

# Passo 1: Amostragem por repetição (Bootstrap)
def bootstrap_sampling(dados, num_amostras):
    amostras = [] #cria uma lista para receber dataframes com as amostras
    for _ in range(num_amostras): # repetir X vezes onde x = argumento num_amostras
        amostra = dados.sample(frac=1, replace=True) #cria um dataframe de mesmo tamanho que dados onde cada linha é um elemento aleatório de dados, podendo repetir elemento
        amostra.reset_index(drop = True, inplace= True)#reordena o index
        amostras.append(amostra)#inclui o df criado na lista amostras
    return amostras

# Passo 2: Seleção aleatória das variáveis (Feature Selection)
# vamos pegar os subconjuntos criados no passo 1 e selecionar aleatoriamente as variaveis/colunas que ele possuirá
def selecao_variaveis(amostras,df_treino):
    amostras_final =[]
    n = round(np.sqrt(df_treino.shape[1]-1))# número  de colunas aleatórias que cada subconjunto deve ter, desconsiderando que uma das colunas do df_treino é y
    for amostra in amostras: # para cada subconjunto...
            var_escolhidas = list(np.random.choice(df_treino.columns[:-1], size = n, replace=False)) #cria aleatoriamente uma lista com n variaveis únicas (replace=False) a partir do df passado no argumento
            amostras_final.append(amostra[var_escolhidas + ['y']])# adiciona à lista o subconjunto somente com as variáveis escolhidas aleatoriamente e y
    return amostras_final

# Passo 3: Criação das árvores
def criar_arvores(amostras_final):
    arvores = []
    for amostra in amostras_final:# Procedimento irá repetir cada df em amostras_final
        X = amostra.iloc[:, :-1]  # Seleciona as Características (x) do df -assumindo que a última coluna do df sempre será a coluna y
        y = amostra.iloc[:, -1]   # Seleciona os Rótulos (y) do df - assumindo que a última coluna do df sempre será a coluna y
        arvore = DecisionTreeClassifier() # cria um modelo de árvore de classificação
        arvore.fit(X, y) # treina o modelo
        arvores.append(arvore) #adiciona  o modelo a lista 
    return arvores

# Passo 4: Agregação
def agregar_arvores(arvores, df_teste):
    
    predicoes=[] #lista para as predições
    i = 0 #contador para iterar amostras_final
    for arvore in arvores: # para cada arvore treinada
        vars= list(amostras_final[i].columns[:-1]) # nome  das variáveis que a arvore[i] foi treinada ([:-1] retira o y)
        x_teste = df_teste[vars] #seleciona no df_teste somente as variáveis que foram usadas no treino da árvore
        predicao =  arvore.predict(x_teste) #faz a predição de y dado x_teste
        predicoes.append(predicao) #inclui a predição na lista predicoes
        i = i + 1

    df_predicoes = pd.DataFrame(predicoes).T #transforma os arrays em dataframe e transpõe as linhas em colunas de modo que cada coluna do array seja a predição de um dos modelos
    colunas = [ 'Y Predito Modelo '+ str(i) for i in range(1, df_predicoes.shape[1] +1 )] # Cria nomes para cada uma das colunas do dataframe
    df_predicoes.columns = colunas # renomeia as colunas com os nomes criados
    #o resultado final das predições será o valor/categoria que foi predito mais vezes pelos modelos (ou seja a moda das linhas do df_predicoes)
    df_resultados = df_predicoes.iloc[:,2:].mode(axis=1).rename(columns = {0: 'y_previsto_final'}) 
    
   
    return df_resultados


**Aplicando as funções**

In [5]:
# Passo 1: Amostragem por repetição (Bootstrap)
amostras = bootstrap_sampling(df_treino, 5)

In [6]:
#Exemplos de amostra geradas
amostras[0]

,x1,x2,x3,x4,x5,x6,y
0,9,7,3,4,1,1,0
1,3,8,7,3,6,10,1
2,3,8,7,3,6,10,1
3,3,8,7,3,6,10,1
4,9,3,3,6,10,4,0


In [7]:
# Passo 2: Seleção aleatória das variáveis (Feature Selection)
# vamos pegar os subconjuntos criados no passo 1 e selecionar aleatoriamente as variaveis/colunas que ele possuirá
amostras_final = selecao_variaveis(amostras,df_treino)

In [8]:
#Exemplos de amostra final_geradas
amostras_final[0]

,x4,x3,y
0,4,3,0
1,3,7,1
2,3,7,1
3,3,7,1
4,6,3,0


In [9]:
# Passo 3: Criação das árvores
arvores = criar_arvores(amostras_final)
arvores

[DecisionTreeClassifier(),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(),
 DecisionTreeClassifier()]

In [10]:
# Passo 3: Agregação
agregar_arvores(arvores, df_teste)


,y_previsto_final
0,1
1,0
2,1
